In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
import re
def readFastaFile(Fasta): ## Working fine
    handle=open(Fasta,'r')
    lines=[]
    for line in handle:
        if('>' not in line):
            lineRead = re.split("\n",line)
            lines.append(lineRead[0])
            
    handle.close()
    
    return lines

In [159]:
h3k4me3=readFastaFile('HCT116_H3K4me3_GRCh38.fasta')
h3k4me1=readFastaFile('HCT116_H3K4me1_GRCh38.fasta')

In [160]:
len(h3k4me3) ## 60741
#len(h3k4me1) ## 102094

60741

In [161]:
 import random
h3k4me3_random = random.sample(h3k4me3, 1000)
h3k4me1_random = random.sample(h3k4me1, 1000)


In [162]:
## From code above, selecting a random subset of h3k4me3 and h3k4me1 to avoid bias. Will use this subset
## to build training and test set

In [163]:
h3k4me3_label = [0] * 1000
h3k4me1_label = [1] * 1000

In [164]:
h3k4me3_random.extend(h3k4me1_random)

In [165]:
len(h3k4me3_random)

2000

In [166]:
h3k4me3_label.extend(h3k4me1_label)

In [167]:
len(h3k4me3_label)

2000

In [168]:
h3k4me3_random

['...',
 ...]

In [240]:
def encode_dna(sequence):
    """Encodes DNA sequences to one-hot integer representation."""
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3,'a': 0, 'c': 1, 'g': 2, 't': 3,'N': 4}
    return np.array([mapping[base] for base in sequence])

In [241]:
# 2. Encode the sequences
encoded_sequences = np.array([encode_dna(seq) for seq in h3k4me3_random])

/var/folders/y3/j7m02tjx5tl14byfj3dnk__00000gn/T/ipykernel_9813/4288198879.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  encoded_sequences = np.array([encode_dna(seq) for seq in h3k4me3_random])


In [242]:
encoded_sequences[1:10]

array([array([3, 2, 1, 0, 1, 3, 1, 1, 2, 0, 2, 1, 1, 0, 1, 0, 1, 0, 1, 3, 1, 0,
              1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 3, 2, 2, 1, 1, 1, 1, 0, 2,
              0, 1, 1, 1, 3, 2, 1, 3, 1, 3, 1, 1, 0, 1, 0, 2, 0, 0, 0, 3, 0, 1,
              3, 2, 1, 0, 1, 0, 1, 1, 1, 3, 2, 1, 3, 1, 3, 1, 1, 0, 1, 0, 2, 0,
              0, 0, 3, 0, 0, 3, 1, 0, 2, 2, 1, 0, 3, 0, 1, 3, 1, 0, 2, 3, 1, 1,
              1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 2, 2, 1, 0, 3, 0, 1, 0, 3, 0,
              1, 0, 3, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 3, 2, 3, 0, 1, 1, 1, 1,
              0, 3, 2, 3, 0, 1, 0, 3, 1, 1, 0, 3, 0, 3, 2, 1, 1, 0, 3, 0, 0, 1,
              1, 0, 2, 0, 1, 0, 1, 0, 3, 1, 3, 1, 3, 1, 3, 1, 0, 3, 0, 1, 0, 2,
              0, 0, 0, 1, 0, 1, 0, 2, 0, 1, 0, 3, 2, 3, 3, 1, 0, 2, 3, 1, 0, 1,
              0, 3, 2, 3, 2, 1, 0, 3, 2, 3, 1, 1, 3, 3, 1, 2, 0, 3, 0, 1, 1, 1,
              0, 2, 1, 0, 3, 1, 1, 1, 0, 0, 1, 0, 1, 0, 2, 0, 0, 1, 0, 1, 3, 1,
              1, 3, 0, 2, 2, 1, 0, 1, 0,

In [243]:
# 3. Padding sequences to make them uniform
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = max(len(seq) for seq in encoded_sequences)
encoded_sequences = pad_sequences(encoded_sequences, maxlen=max_length, padding='post')

In [244]:
encoded_sequences[1:100]

array([[3, 2, 1, ..., 0, 0, 0],
       [3, 1, 3, ..., 0, 0, 0],
       [3, 1, 3, ..., 0, 0, 0],
       ...,
       [2, 1, 3, ..., 0, 0, 0],
       [3, 1, 3, ..., 0, 0, 0],
       [0, 3, 2, ..., 0, 0, 0]], dtype=int32)

In [246]:
# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(encoded_sequences, h3k4me3_label, test_size=0.2, random_state=42)

In [247]:
# Convert labels to categorical (one-hot encoding for multiclass)
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

In [248]:
# 5. Define the CNN model
model = Sequential([
    Embedding(input_dim=5, output_dim=10, input_length=max_length),  # 4 nucleotides -> 8-d embedding
    Conv1D(filters=128, kernel_size=3, activation='sigmoid'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes (binary classification)
])

In [249]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [253]:
len(y_train)

1600

In [251]:
len(X_train)

1600

In [252]:
# 7. Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_split=0.2)

Epoch 1/30
80/80 [==============================] - 40s 492ms/step - loss: 24.9952 - accuracy: 0.4898 - val_loss: 0.6928 - val_accuracy: 0.5437
Epoch 2/30
80/80 [==============================] - 39s 486ms/step - loss: 0.6932 - accuracy: 0.5008 - val_loss: 0.6935 - val_accuracy: 0.4563
Epoch 3/30
80/80 [==============================] - 39s 486ms/step - loss: 0.6931 - accuracy: 0.5117 - val_loss: 0.6941 - val_accuracy: 0.4563
Epoch 4/30
80/80 [==============================] - 38s 476ms/step - loss: 0.6930 - accuracy: 0.5117 - val_loss: 0.6946 - val_accuracy: 0.4563
Epoch 5/30
80/80 [==============================] - 39s 489ms/step - loss: 0.6930 - accuracy: 0.5117 - val_loss: 0.6949 - val_accuracy: 0.4563
Epoch 6/30
80/80 [==============================] - 38s 481ms/step - loss: 0.6930 - accuracy: 0.5117 - val_loss: 0.6946 - val_accuracy: 0.4563
Epoch 7/30
80/80 [==============================] - 39s 482ms/step - loss: 0.6929 - accuracy: 0.5117 - val_loss: 0.6950 - val_accuracy: 0.456

In [254]:
len(X_train) ## 8000
len(y_train)

1600

In [255]:
# 8. Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

13/13 [==============================] - 3s 206ms/step - loss: 0.6934 - accuracy: 0.4975
Test Loss: 0.6934419870376587
Test Accuracy: 0.4975000023841858


In [261]:
y_test

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.